Analyse age gap between actors and actresses who play love interests in the big screen. 

1. Find pairs of characters who are in a relationship
2. Get age of actors and actresses when they played the given character

In [1]:
import sys
sys.path.append("../../")

import bechdelai.data.wikipedia as wiki
import spacy
import outputformat as ouf
import pandas as pd
import requests
import io

import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

from pathlib import Path

# 1. Find love interests 
Use plot text from Wikipedia to find characters who are in a relationship.

In [2]:
movies = ['Call Me by Your Name (film)', 'The Holiday','Fantastic Beasts: The Secrets of Dumbledore','Beauty and the Beast (2017 film)']
plots = {}
for movie in movies:
    sections = wiki.get_section_text(movie,['Plot'])
    plots[movie]= sections['Plot']
    ouf.boxtitle(movie)
    print(plots[movie])

╭─────────────────────────────╮
│ Call Me by Your Name (film) │
╰─────────────────────────────╯
It is the summer of 1983. Elio, a 17-year-old Jewish Italo-French young man, lives with his parents in rural Northern Italy. Elio's father, a professor of archaeology, invites a 24-year-old American graduate student, Oliver, who is also Jewish, to live with the family over the summer and help with his academic paperwork. Elio, an introspective bibliophile and a talented musician, initially thinks he has little in common with Oliver, who appears confident and carefree. Elio spends much of the summer reading, playing piano, and hanging out with his childhood friends, Chiara and Marzia. During a volleyball match, Oliver touches Elio's back as a sign of interest but Elio brushes it off. However, Elio later finds himself jealous upon seeing Oliver pursue Chiara.
Elio and Oliver spend more time together, going for long walks into town, and accompanying Elio's father on an archaeological trip. Elio

╭──────────────────────────────────╮
│ Beauty and the Beast (2017 film) │
╰──────────────────────────────────╯
An enchantress disguised as an old beggar woman arrives at a castle during a ball and offers the host, a cruel and selfish prince, an enchanted rose in return for shelter from a storm. When he refuses the offer, the enchantress reveals her true form and transforms the prince into a beast and his servants into household objects before erasing the castle, himself, and his servants from the memories of their loved ones and everyone else in the town. She warns the prince that the curse will be broken, only if he can learn to love another and have that person love him in return by the time the last petal of the rose falls, or else he will remain a beast for the rest of his life while his servants become inanimate.
Several years later, in the small town of Villeneuve, Belle, the book-loving daughter of a widowed artist named Maurice, dreams of adventure. She constantly brushes off a

In [3]:
nlp = spacy.load("en_core_web_md")
for movie in plots.keys():
    doc = nlp(plots[movie])
    displacy.render(doc,style="ent")

/home/juliana/anaconda3/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_md' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
def find_previous_sent(doc,sent):
    if sent.start==0:
        return ''
    for s in doc.sents: # check if it's possible to optimize
        if s.end==sent.start:
            return s
    print('No sentence was found')    

matcher = PhraseMatcher(nlp.vocab)
terms = ["have sex", "sleep together","marry","love","kiss","date"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

for movie in plots.keys():
    ouf.boxtitle(movie)
    text = plots[movie]
    doc = nlp(text)
    matches = matcher(doc) # [lexeme, start token, end token]
    for match_id, start, end in matches:
        # create the matched span and assign the match_id as a label
        span = Span(doc, start, end, label=match_id)
        previous_span = find_previous_sent(doc,span.sent)
        print(previous_span,span.sent)
        displacy.render(span.sent,"dep")
#         svg = displacy.render(span.sent,style = "dep",jupyter=False)
#         output_path = Path("dep_{}_{}.svg".format(movie,start)) 
#         output_path.open("w", encoding="utf-8").write(svg)
    

        who = []
        who_pos = []
        for ent in span.sent.ents:
            if ent.label_=="PERSON":
                who.append(ent.text)
        if len(who)<2:
            if len(previous_span):
                for ent in previous_span.ents:
                    if ent.label_=="PERSON":
                        who.append(ent.text)
                
        print(who)


╭─────────────────────────────╮
│ Call Me by Your Name (film) │
╰─────────────────────────────╯
Elio eventually confesses his feelings to Oliver, who is awestruck and tells him they cannot discuss such things. Later, in a secluded spot, the two kiss for the first time.


['Elio', 'Oliver']
Oliver is reluctant to take things further and they do not speak for several days. 
Elio goes on a date with Marzia and the two have sex.


['Elio', 'Marzia']
Oliver is reluctant to take things further and they do not speak for several days. 
Elio goes on a date with Marzia and the two have sex.


['Elio', 'Marzia']
Oliver writes back, asking Elio to meet him at midnight. Elio agrees and they sleep together for the first time.


['Elio', 'Oliver', 'Elio']
╭─────────────╮
│ The Holiday │
╰─────────────╯
 Iris Simpkins, a society columnist for The Daily Telegraph in London, is still madly in love with her ex, Jasper Bloom, despite the fact that he cheated on her and is emotionally needy.


...

['Jacob', 'Queenie', 'Tina Goldstein']
╭──────────────────────────────────╮
│ Beauty and the Beast (2017 film) │
╰──────────────────────────────────╯
When he refuses the offer, the enchantress reveals her true form and transforms the prince into a beast and his servants into household objects before erasing the castle, himself, and his servants from the memories of their loved ones and everyone else in the town. She warns the prince that the curse will be broken, only if he can learn to love another and have that person love him in return by the time the last petal of the rose falls, or else he will remain a beast for the rest of his life while his servants become inanimate.


[]
When he refuses the offer, the enchantress reveals her true form and transforms the prince into a beast and his servants into household objects before erasing the castle, himself, and his servants from the memories of their loved ones and everyone else in the town. She warns the prince that the curse will be broken, only if he can learn to love another and have that person love him in return by the time the last petal of the rose falls, or else he will remain a beast for the rest of his life while his servants become inanimate.


[]

Several years later, in the small town of Villeneuve, Belle, the book-loving daughter of a widowed artist named Maurice, dreams of adventure. She constantly brushes off advances from Gaston, an arrogant war veteran as he is not the type of man Belle wishes to marry.


['Maurice']
The Beast then dies just as the last petal falls and the servants become inanimate objects themselves shortly after the ordeal. 
As Belle tearfully professes her love to the Beast, Agathe reveals herself as the enchantress and undoes the curse which repairs the crumbling castle, revives the Beast and restores him and his servants to their human forms and to the villagers' memories once again.


['Agathe']


## Next steps
We are able to extract sentences that can help define the characters' relationship. Next step consists of extracting characters' names in a relationship from these sentences.

Challenges:
- Different structures as observed in all previous exemples
- Proper noums may be missing from sentence (ex: *Later, in a secluded spot, __the two__ kiss for the first time.*) -> consider previous sentence
- There might be other proper noums in the sentence from a character who in not in the relationship (ex: *__Jacob__ and __Queenie__ marry in the former's bakery in New York City, with most of the group and __Tina Goldstein__ in attendance.*) -> how to choose

# 2. Get age of actors and actresses 
Use TMDb data to relate each actor-ress to a character and to compute hir/her age at movie release date

In [5]:
# To get movies data from TMDb, we need to find the movie's ID.

from datetime import datetime
import bechdelai.data.tmdb as tmdb
from bechdelai.data.display import show_movie_suggestions_get_id

query = 'Beauty and the Beast'
data = tmdb.search_movie_from_query(query)
suggestions = tmdb.format_results_for_suggestion(data)
movie_id = show_movie_suggestions_get_id(suggestions, top=3, verbose=True)

,Beauty and the Beast (1991)


,Beauty and the Beast (2017)


,Beauty and the Beast (2009)



Select wanted index:0
ID of the movie: 10020


In [6]:
suggestions[0][1]

10020

In [7]:
def age(birthdate,release_date):
    # Difference in years 
    year_difference = release_date.year - birthdate.year
    
    # Check if birthday happened before or after release date
    one_or_zero = ((release_date.month, release_date.day) < (birthdate.month, birthdate.day))
        
    # If release before birthday -> substract 1
    # If release after birthday -> substract 0
    age = year_difference - one_or_zero
        
    return age

In [8]:
# get the cast data and relase date for a given movie
movies_tmdb_id = [398818,1581,338953,321612] # ids corresponding to movies
movie_id = movies_tmdb_id[0]

release_date = tmdb.get_movie_details_from_id(movie_id)['release_date']
release_date = datetime.strptime(release_date, '%Y-%m-%d')

data = tmdb.get_movie_cast_from_id(movie_id)
cast_df = pd.DataFrame(data["cast"])
cast_df = cast_df[cast_df['known_for_department']=='Acting'][['gender','id','name','character']]



In [9]:
# compute age at release date for each cast member
cast_df.insert(4,'age_at_release',None)
for i,row in cast_df.iterrows():
    birthday = tmdb.get_person_details_from_id(row['id'])['birthday']
    try:
        cast_df.loc[i,'age_at_release'] = age(datetime.strptime(birthday, '%Y-%m-%d'),release_date)
    except TypeError:
        cast_df.loc[i,'age_at_release'] = None
cast_df

,gender,id,name,character,age_at_release
0,2,1190668,Timothée Chalamet,Elio Perlman,21
1,2,53807,Armie Hammer,Oliver,31
2,2,72873,Michael Stuhlbarg,Samuel Perlman,49
3,1,20577,Amira Casar,Annella Perlman,46
4,1,935096,Esther Garrel,Marzia,26
5,1,1592949,Victoire Du Bois,Chiara,29
6,1,1789600,Vanda Capriolo,Mafalda,None
7,2,1789602,Antonio Rimoldi,Anchise,97
8,1,1526272,Elena Bucci,Bambi,None
9,0,1789603,Marco Sgrosso,Nico,None


# Truth from Hollywood Age Gap

In [10]:
url = 'https://hollywoodagegap.com/movies.csv'
url_data = requests.get(url, allow_redirects=True).content
couples_hollywood = pd.read_csv(io.StringIO(url_data.decode('utf-8')))

In [11]:
couples_hollywood[couples_hollywood['Movie Name']=="Call Me by Your Name"]

,Movie Name,Release Year,Director,Age Difference,Actor 1 Name,Actor 1 Gender,Actor 1 Birthdate,Actor 1 Age,Actor 2 Name,Actor 2 Gender,Actor 2 Birthdate,Actor 2 Age
497,Call Me by Your Name,2017,Luca Guadagnino,9,Armie Hammer,man,1986-08-28,31,Timothée Chalamet,man,1995-12-27,22


In [19]:
movies = couples_hollywood['Movie Name'].unique()
for movie in movies[:10]:
    sections = wiki.get_section_text(movie,['Plot'])
    
    data = tmdb.search_movie_from_query(movie)
    suggestions = tmdb.format_results_for_suggestion(data)
    movie_id = suggestions[0][1]
    
    tmdb_data = tmdb.get_movie_cast_from_id(movie_id)
    cast_df = pd.DataFrame(tmdb_data["cast"])
    cast_df = cast_df[cast_df['known_for_department']=='Acting'][['gender','id','name','character']]

    
    ouf.boxtitle(movie)
    try:
        print(sections['Plot'])
    except KeyError:
        pass
    if len(cast_df):
        print(cast_df)

╭──────────────────╮
│ Harold and Maude │
╰──────────────────╯
Harold Chasen is a 19-year-old man obsessed with death. He stages elaborate fake suicides, attends funerals (usually for people that he doesn't know), and drives a hearse, all to the chagrin of his self-obsessed, wealthy socialite mother. His mother sends Harold to a psychoanalyst, sets him up with blind dates, and buys him a luxury car, all schemes he subverts in his own way.
Harold meets 79-year-old Maude one day while at a random stranger's funeral mass, and find they share a hobby. Harold is entranced by Maude's quirky outlook on life, which is bright and delightfully carefree in contrast with his moribund demeanor. Maude lives in a decommissioned railroad car and thinks nothing of breaking the law; she is quite skilled at stealing cars and will swiftly uproot an ailing tree on public property to re-plant it in the forest. She and Harold form a bond and Maude shows Harold the pleasures of art and music (including how to

╭──────────────────╮
│ The Big Lebowski │
╰──────────────────╯
In the early 1990s, Los Angeles slacker Jeffrey "The Dude" Lebowski is assaulted in his home by two enforcers for porn kingpin Jackie Treehorn, who is owed money by the wife of a different Jeffrey Lebowski.  One of the goons urinates on the Dude's favorite rug before they realize they have the wrong man and leave.
Advised by his bowling partners, Vietnam veteran Walter Sobchak and Donny Kerabatsos, the Dude visits wealthy philanthropist Jeffrey ("Big") Lebowski, demanding compensation for the rug. Big refuses, but the Dude tricks Big's assistant Brandt into letting him take a similar rug from the mansion. Outside he meets Bunny, Big's trophy wife, and her German nihilist friend Uli.  Soon after this, Bunny is apparently kidnapped and Big hires The Dude to deliver the requested ransom money, one million dollars. That night, a different pair of thugs accost the Dude, taking his replacement rug on behalf of Big's daughter from

╭────────────╮
│ Poison Ivy │
╰────────────╯
    gender       id                name       character
0        1    69597      Drew Barrymore             Ivy
1        1     3282        Sara Gilbert   Sylvie Cooper
2        2     4139        Tom Skerritt   Darryl Cooper
3        1    36169         Cheryl Ladd  Georgie Cooper
4        2     6193   Leonardo DiCaprio             Guy
5        1  1219601       Jeanne Sakata        Isabelle
6        2    85421        Time Winters         Old Man
7        0  1217488          Alan Stock             Bob
8        0  2461564          E.J. Moore             Kid
9        0  1236477           J.B. Quon     Another Kid
10       2  3247418     Michael Goldner      Man in Car
11       2  1336663     Charley Hayward            Tiny
12       2  3098995          Billy Kane           James
13       2   172400       Tony Ervolina   Man on Screen
14       1  1190638  Mary Gordon Murray            M.D.
15       1  3247423      Charla Sampsel         Orderly
16 

╭────────────────────╮
│ Husbands and Wives │
╰────────────────────╯
The film is about two couples: Jack (Sydney Pollack) and Sally (Judy Davis), and Gabe (Woody Allen) and Judy (Mia Farrow). The film starts when Jack and Sally arrive at Gabe and Judy's apartment and announce their separation. Gabe is shocked, but Judy takes the news personally and is very hurt. Still confused, they go out for dinner at a Chinese restaurant.
A few weeks later Sally goes to the apartment of a colleague. They plan to go out together to the opera and then to dinner. Sally asks if she can use his phone, and calls Jack. Learning from him that he has met someone, she accuses him of having had an affair during their marriage.
Judy and Gabe are introduced to Jack's new girlfriend, Sam (Lysette Anthony), an aerobics trainer. While Judy and Sam shop, Gabe calls Jack's new girlfriend a "cocktail waitress" and tells him that he is crazy for leaving Sally for her. About a week later, Judy introduces Sally to Michae